In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset

In [3]:
data_files = {"../dataset_jit/test_jit.csv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 5000 examples [00:00, 115017.06 examples/s]


In [23]:
dataset = dataset['train'].select(range(1000))

In [25]:
dataset

Dataset({
    features: ['ko', 'je'],
    num_rows: 1000
})

In [26]:
dataset['ko'][0]

'그래도 그거 살살 떠서 먹지 . 그거 안 먹진 않아 . 어떻게 옛날 삶이야 삶이 었어'

In [10]:
import torch
from tqdm import tqdm
import numpy as np
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('hyunwoongko/kobart')
model_path = "./result/bleu/checkpoint-11000"
bart_bleu = BartForConditionalGeneration.from_pretrained(model_path)

model_path = "./result/comet/checkpoint-3000"
bart_comet = BartForConditionalGeneration.from_pretrained(model_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
bart_bleu.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [14]:
input_text = dataset['train']['ko'][0]
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

decoded_ids = bart_bleu.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
output = tokenizer.decode(decoded_ids[0], skip_special_tokens=True)

print(f'output : {output}')

=: 경헤도 그거 ᄉᆞᆯᄉᆞᆯ 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶사 삶이 엇어.


In [15]:
input_text = dataset['train']['ko'][0]
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

decoded_ids = bart_comet.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
output = tokenizer.decode(decoded_ids[0], skip_special_tokens=True)

print(f'output : {output}')

=: 게도 그거 살살 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶이라 삶이 엇어.


In [27]:
# 입력 텍스트와 참조 번역 텍스트 분리
input_texts = [item for item in dataset['ko']]
reference_texts = [item for item in dataset['je']]

In [33]:
def calculate_accuracy(predictions, references):
    total_words = 0
    correct_words = 0
    
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        total_words += len(ref_tokens)
        correct_words += sum(p == r for p, r in zip(pred_tokens, ref_tokens))
    
    accuracy = correct_words / total_words if total_words > 0 else 0
    return accuracy

def calculate_char_accuracy(predictions, references):
    total_chars = 0
    correct_chars = 0
    
    for pred, ref in zip(predictions, references):
        pred_chars = list(pred)
        ref_chars = list(ref)
        total_chars += len(ref_chars)
        correct_chars += sum(p == r for p, r in zip(pred_chars, ref_chars))
    
    accuracy = correct_chars / total_chars if total_chars > 0 else 0
    return accuracy

In [35]:
from tqdm import tqdm

# 모델 예측 수행
predictions_bleu = []
for input_text in tqdm(input_texts):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_bleu.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predictions_bleu.append(summary_text)

100%|██████████| 1000/1000 [12:25<00:00,  1.34it/s]


In [36]:
# 정확도 계산
accuracy = calculate_accuracy(predictions_bleu, reference_texts)
print(f'BLEU Accuracy: {accuracy:.2%}')

# 정확도 계산
char_accuracy = calculate_char_accuracy(predictions_bleu, reference_texts)
print(f'Character-level Accuracy: {char_accuracy:.2%}')

BLEU Accuracy: 19.73%
Character-level Accuracy: 26.21%


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset

In [ ]:
data_files = {"../dataset_jit/test_jit.csv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

Generating train split: 5000 examples [00:00, 115017.06 examples/s]


In [ ]:
dataset = dataset['train'].select(range(1000))

In [ ]:
dataset

Dataset({
    features: ['ko', 'je'],
    num_rows: 1000
})

In [ ]:
dataset['ko'][0]

'그래도 그거 살살 떠서 먹지 . 그거 안 먹진 않아 . 어떻게 옛날 삶이야 삶이 었어'

In [ ]:
import torch
from tqdm import tqdm
import numpy as np
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('hyunwoongko/kobart')
model_path = "./result/bleu/checkpoint-11000"
bart_bleu = BartForConditionalGeneration.from_pretrained(model_path)

model_path = "./result/comet/checkpoint-3000"
bart_comet = BartForConditionalGeneration.from_pretrained(model_path)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
bart_bleu.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
input_text = dataset['train']['ko'][0]
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

decoded_ids = bart_bleu.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
output = tokenizer.decode(decoded_ids[0], skip_special_tokens=True)

print(f'output : {output}')

=: 경헤도 그거 ᄉᆞᆯᄉᆞᆯ 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶사 삶이 엇어.


In [ ]:
input_text = dataset['train']['ko'][0]
inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)

decoded_ids = bart_comet.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
output = tokenizer.decode(decoded_ids[0], skip_special_tokens=True)

print(f'output : {output}')

=: 게도 그거 살살 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶이라 삶이 엇어.


In [ ]:
# 입력 텍스트와 참조 번역 텍스트 분리
input_texts = [item for item in dataset['ko']]
reference_texts = [item for item in dataset['je']]

In [ ]:
from tqdm import tqdm

# 모델 예측 수행
predictions_bleu = []
for input_text in tqdm(input_texts):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_bleu.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predictions_bleu.append(summary_text)

100%|██████████| 1000/1000 [12:25<00:00,  1.34it/s]


In [ ]:
# 정확도 계산
accuracy = calculate_accuracy(predictions_bleu, reference_texts)
print(f'BLEU Accuracy: {accuracy:.2%}')

# 정확도 계산
char_accuracy = calculate_char_accuracy(predictions_bleu, reference_texts)
print(f'Character-level Accuracy: {char_accuracy:.2%}')

BLEU Accuracy: 19.73%
Character-level Accuracy: 26.21%


In [44]:
for idx in range(10):
    print(f"input  :     {dataset['ko'][idx]}")
    print(f"translation :{predictions_bleu[idx]}")
    print(f"target :     {reference_texts[idx]}\n")
    


input  :     그래도 그거 살살 떠서 먹지 . 그거 안 먹진 않아 . 어떻게 옛날 삶이야 삶이 었어
translation :경헤도 그거 ᄉᆞᆯᄉᆞᆯ 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶사 삶이 엇어.
target :     경헤도 거 ᄉᆞᆯᄉᆞᆯ 거려뒁 먹주 . 거 안 먹진 안허여 . 어떵 옛날 삶사 삶 이랏어

input  :     마찬가지 그렇게 쌀 넣어서 물 넣고 해서 삶아가면 익어서 죽 되는 거 .
translation :마찬가지 경 ᄊᆞᆯ  물 놓곡 헹 ᄉᆞᆱ아가민 익엉 죽 뒈는 거.
target :     마찬가지 겅 ᄊᆞᆯ 넣어근엥 물 넣곡 허영 ᄉᆞᆱ아가면 익어근엥 죽 뒈는 거

input  :     그것보고 저 거시기 , 부엌 .
translation :그것ᄀᆞ라 저 거시기, 정지.
target :     그것ᄀᆞ란 저 거세기 , 정지 .

input  :     예 , 동김치 그거 .
translation :예, 동짐치 그거.
target :     예 , 동김치 . 그거 .

input  :     그렇게 해버리니까 많이 김매야겠군 요
translation :경 헤부난 하영 검질메사켜예?
target :     겅 헤부난 많이 검질메야겠다예 .

input  :     그때도 가면 그땐 그렇게해서 놀았어 . 나 그거 잊어버리지 않아 . 여기 넷 다섯 , 저쪽에 넷 다섯 서면 , 딱 손 이어서 저쪽으로 가면서 무슨 사꾸라또 만또민또 하나이세 몸메 하면서 아이 하나 가서 딱 지키면 그 아이 하나 잡아오고 또 그쪽에서 와서 그렇게 하고 . 그렇게 하면서도 놀고 . 그저 밤에 진또리 .
translation :그때도 가민 그땐 경헹 놀앗어. 나 그거 잊어불지 안허여. 이디 넷 다섯, 저쪽에 넷 다섯 사민, 딱 손 이엉 저쪽으로 가멍 무신 사꾸라
target :     그때도 가민 그땐 경 헨 놀앗어 . 나 그거 잊어불지 안 허여 . 이디 넷 다섯 저짝에 넷 다섯 사민 , 딱 손 이어근에 저레 가멍 무신 사꾸라또 민또만또 하

In [ ]:
bart_comet.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [ ]:
predictions = []
for input_text in tqdm(input_texts):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_comet.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predictions.append(summary_text)

# 정확도 계산
accuracy = calculate_accuracy(predictions, reference_texts)
print(f'COMET Accuracy: {accuracy:.2%}')


100%|██████████| 1000/1000 [12:14<00:00,  1.36it/s]

COMET Accuracy: 19.07%


In [ ]:
char_accuracy = calculate_char_accuracy(predictions, reference_texts)
print(f'Character-level Accuracy: {char_accuracy:.2%}')

Character-level Accuracy: 25.04%


In [37]:
predictions_bleu

['경헤도 그거 ᄉᆞᆯᄉᆞᆯ 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶사 삶이 엇어.',
 '마찬가지 경 ᄊᆞᆯ  물 놓곡 헹 ᄉᆞᆱ아가민 익엉 죽 뒈는 거.',
 '그것ᄀᆞ라 저 거시기, 정지.',
 '예, 동짐치 그거.',
 '경 헤부난 하영 검질메사켜예?',
 '그때도 가민 그땐 경헹 놀앗어. 나 그거 잊어불지 안허여. 이디 넷 다섯, 저쪽에 넷 다섯 사민, 딱 손 이엉 저쪽으로 가멍 무신 사꾸라',
 '표선리란 마을이예 관광도로의 요충지가 됩니다, 여기가.',
 '청묵도 ᄆᆞ물ᄏᆞ루, 그냥 묵도 ᄆᆞ물ᄏᆞ루.',
 '그것이 구강이라. 그디 영 놧당 이젠 삼월 나가민 요 방만이만 예를 들엉양 방만이만 놓아, 씨를. 씨를 놓민 그것이 커 가민 그때도 오줌도 주곡 무시',
 '빙아리 깨왕 병아리 크민.',
 '줄기리 이신 건 조로 무끄곡, 줄리 엇인 건 께로 틀엉 새로 께 틀엉 무끄곡.',
 '막 공들영 경 멘드는.',
 '오누이가 학교를 다녀왓는데 그걸 친 걸 말 안헌 거라.',
 '마늘은, 그 온전허게 뒌 마늘 헐 때 지시 담으민 그거 지시 담갓당 이제, 다 안 먹어지민 무시거 저 고기 보끌 때, 바당궤기 보끌 때나',
 '이제 을미년 허민 팔십 둘  거주게.',
 '으. 대개 서모커리를 부엌을 헤마씨. 동모커리, 서모커리는 소외양간을 아녀거든. 왜?',
 '게난 ᄆᆞᆯ테엔도 헤지주. ᄆᆞᆯ테엔도 헤지주게.',
 '그거 햇볕에 놓민 안 뒈지예?',
 '걸름허고 또 다시 우리가 음식물 찌거기 다 도새기가 먹으난 아무것도 버릴 것이 엇주게.',
 '옛날엔 처음엔 어린 땐 거 봉가왓는디 철든 후제 어이구 저 늙은 어른덜 ᄀᆞᆮ는 거 꿩의 가슴 매의 가슴 허난 아이고 얼마나 저 새끼 잃으민 애석',
 '그 동글락 쉐에다 가 자전거 바퀴 식으로 이렇게 헤서 만들어가지고 쉐를 이렇게 구부려서 글로 헤서 쭉 허게 돌아뎅기멍 허는 거.',
 'ᄑᆞᆺ도 놓고 경 아녀민 또 이제 무신 무수 썰어놔도 뒌다.',
 'ᄑᆞᆺ 저 영 헤근에 영 물에 ᄒᆞ꼼 컷당.',
 '

In [30]:
bart_comet.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), 

In [31]:
predictions = []
for input_text in tqdm(input_texts):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_comet.generate(inputs["input_ids"], num_beams=4, max_length=50, early_stopping=True)
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predictions.append(summary_text)

# 정확도 계산
accuracy = calculate_accuracy(predictions, reference_texts)
print(f'COMET Accuracy: {accuracy:.2%}')


100%|██████████| 1000/1000 [12:14<00:00,  1.36it/s]

COMET Accuracy: 19.07%


In [34]:
char_accuracy = calculate_char_accuracy(predictions, reference_texts)
print(f'Character-level Accuracy: {char_accuracy:.2%}')

Character-level Accuracy: 25.04%


In [46]:
for idx in range(10):
    print(f"input  : {dataset['ko'][idx]}")
    print(f"comet  : {predictions[idx]}")
    print(f"bleu   : {predictions_bleu[idx]}")
    print(f"target : {reference_texts[idx]}\n")

input  : 그래도 그거 살살 떠서 먹지 . 그거 안 먹진 않아 . 어떻게 옛날 삶이야 삶이 었어
comet  : 게도 그거 살살 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶이라 삶이 엇어.
bleu   : 경헤도 그거 ᄉᆞᆯᄉᆞᆯ 거령 먹주. 그거 안 먹진 안허여. 어떵 엿날 삶사 삶이 엇어.
target : 경헤도 거 ᄉᆞᆯᄉᆞᆯ 거려뒁 먹주 . 거 안 먹진 안허여 . 어떵 옛날 삶사 삶 이랏어

input  : 마찬가지 그렇게 쌀 넣어서 물 넣고 해서 삶아가면 익어서 죽 되는 거 .
comet  : 마찬가지 경 ᄊᆞᆯ  물 놓곡 헹 ᄉᆞᆱ아가민 익엉 죽 뒈는 거.
bleu   : 마찬가지 경 ᄊᆞᆯ  물 놓곡 헹 ᄉᆞᆱ아가민 익엉 죽 뒈는 거.
target : 마찬가지 겅 ᄊᆞᆯ 넣어근엥 물 넣곡 허영 ᄉᆞᆱ아가면 익어근엥 죽 뒈는 거

input  : 그것보고 저 거시기 , 부엌 .
comet  : 그것ᄀᆞ라 저 거시기, 부엌.
bleu   : 그것ᄀᆞ라 저 거시기, 정지.
target : 그것ᄀᆞ란 저 거세기 , 정지 .

input  : 예 , 동김치 그거 .
comet  : 예, 동짐치 그거.
bleu   : 예, 동짐치 그거.
target : 예 , 동김치 . 그거 .

input  : 그렇게 해버리니까 많이 김매야겠군 요
comet  : 경 헤부난 하영 김메야겟구나예?
bleu   : 경 헤부난 하영 검질메사켜예?
target : 겅 헤부난 많이 검질메야겠다예 .

input  : 그때도 가면 그땐 그렇게해서 놀았어 . 나 그거 잊어버리지 않아 . 여기 넷 다섯 , 저쪽에 넷 다섯 서면 , 딱 손 이어서 저쪽으로 가면서 무슨 사꾸라또 만또민또 하나이세 몸메 하면서 아이 하나 가서 딱 지키면 그 아이 하나 잡아오고 또 그쪽에서 와서 그렇게 하고 . 그렇게 하면서도 놀고 . 그저 밤에 진또리 .
comet  : 그때도 가민 그땐 경헹 놀앗어. 나 그거 잊어불지 안허여. 이디 넷 다섯, 저쪽에 넷 다섯 서면, 딱 손 이어서 저쪽으로 가멍 무